# Web Traffic Prediction

## Overview

This solution is about predicting the future behaviour of time series’ that describe the web traffic for Wikipedia articles. The data contains about 145k time series and comes in two separate files: train_2.csv holds the traffic data, where each column is a date and each row is an article, and key_2.csv contains a mapping between page names and a unique ID column (to be used in the submission file).

Each of these time series represent a number of daily views of a different Wikipedia article, starting from July, 1st, 2015 up until December 31st, 2016. The leaderboard during the training stage is based on traffic from January, 1st, 2017 up until  September 10th, 2017.

This is a competition hosted on Kaggle, the dataset has been taken from Kaggle.


## Let's Import the Required Libraries.

In [21]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

from keras.layers.normalization import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers 

import keras

from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold
from pathlib import Path
import re

from datetime import timedelta

import warnings
import scipy
from datetime import timedelta

from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.tsatools import lagmat
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

### Submissions are evaluated on Symmetric mean absolute percentage error (SMAPE) between forecasts and actual values.

SMAPE is an accuracy measure based on percentage (or relative) errors. It is usually defined as follows:

SMAPE = \frac{1}{n} \sum\limits_{t=1}^n{\frac{|F_t - A_t|}{F_t + A_t}}

In [2]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def smape2D(y_true, y_pred):
    return smape(np.ravel(y_true), np.ravel(y_pred))

In [3]:
path = Path('../data/web-traffic')
list(path.iterdir())

[PosixPath('../data/web-traffic/key_1.csv'),
 PosixPath('../data/web-traffic/key_2.csv'),
 PosixPath('../data/web-traffic/sample_submission_1.csv'),
 PosixPath('../data/web-traffic/sample_submission_2.csv'),
 PosixPath('../data/web-traffic/train_1.csv'),
 PosixPath('../data/web-traffic/train_2.csv')]

### About the Data files

- This challenge is about predicting the future behaviour of time series’ that describe the web traffic for Wikipedia articles. The data contains about 145k time series
- The train_1.csv file has data from `2015-07-01` to `2016-12-31`
- The train_2.csv file has data from `2015-07-01` to `2017-09-10`
- We will be using data from train_2.csv for our training. Which had all data from train_1.csv and additional record. It has records from `July, 1st, 2015` to `September 1st, 2017`
- We have to predict daily page views between `September 13th, 2017` to `November 13th, 2017`.
- key_*.csv gives the mapping between the page names and the shortened Id column used for prediction

In [ ]:
df = pd.read_csv(path/"train_1.csv")
df.head()

In [ ]:
train_all = pd.read_csv(path/"train_2.csv")
train_all.head()

In [ ]:
train_all.shape

In [ ]:
train_all.columns

The training data contains prediction of page views for 803 days.

## Initial Data Exploration

- Let's make each page and date into it's individual columns 
- Also we will check if the given day is a weekend or weekday

In [ ]:
train_flattened = pd.melt(train_all[list(train_all.columns[-803:])+['Page']], 
                          id_vars='Page', var_name='date', value_name='Visits')

train_flattened['date'] = train_flattened['date'].astype('datetime64[ns]')
train_flattened['weekend'] = ((train_flattened.date.dt.dayofweek) // 5 == 1).astype(float)

In [ ]:
df_median = pd.DataFrame(train_flattened.groupby(['Page'])['Visits'].median())
df_median.columns = ['median']

df_mean   = pd.DataFrame(train_flattened.groupby(['Page'])['Visits'].mean())
df_mean.columns = ['mean']

df_std    = pd.DataFrame(train_flattened.groupby(['Page'])['Visits'].std())
df_std.columns = ['std']

train_flattened = train_flattened.set_index('Page').join(df_mean).join(df_median).join(df_std)
train_flattened.head()

In [ ]:
train_flattened.reset_index(inplace=True)

## Aggregation & Visualisation

In [ ]:
plt.figure(figsize=(50, 8))
mean_group = train_flattened[['Page', 'date', 'Visits']].groupby(['date'])['Visits'].mean()
plt.plot(mean_group)
plt.title('Time Series - Average')
plt.show()

In [ ]:
plt.figure(figsize=(50, 8))
median_group = train_flattened[['Page', 'date', 'Visits']].groupby(['date'])['Visits'].median()
plt.plot(median_group, color='r')
plt.title('Time Series - Average')
plt.show()

In [ ]:
plt.figure(figsize=(50, 8))
std_group = train_flattened[['Page','date','Visits']].groupby(['date'])['Visits'].std()
plt.plot(std_group, color = 'g')
plt.title('Time Series - std')
plt.show()

#### Let's check if Language has any Impact on Page View

In [ ]:
df_train = train_all.copy()

In [ ]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res[0][0:2]
    return 'na'

df_train['lang'] = df_train.Page.map(get_language)

In [ ]:
lang_sets = {}
lang_sets['en'] = df_train[df_train.lang=='en'].iloc[:,0:-1]
lang_sets['ja'] = df_train[df_train.lang=='ja'].iloc[:,0:-1]
lang_sets['de'] = df_train[df_train.lang=='de'].iloc[:,0:-1]
lang_sets['na'] = df_train[df_train.lang=='na'].iloc[:,0:-1]
lang_sets['fr'] = df_train[df_train.lang=='fr'].iloc[:,0:-1]
lang_sets['zh'] = df_train[df_train.lang=='zh'].iloc[:,0:-1]
lang_sets['ru'] = df_train[df_train.lang=='ru'].iloc[:,0:-1]
lang_sets['es'] = df_train[df_train.lang=='es'].iloc[:,0:-1]

sums = {}
for key in lang_sets:
    sums[key] = lang_sets[key].iloc[:,1:].sum(axis=0) / lang_sets[key].shape[0]

In [ ]:
days = [r for r in range(sums['en'].shape[0])]

fig = plt.figure(1,figsize=[10,10])
plt.ylabel('Views per Page')
plt.xlabel('Day')
plt.title('Pages in Different Languages')
labels={'en':'English','ja':'Japanese','de':'German',
        'na':'Media','fr':'French','zh':'Chinese',
        'ru':'Russian','es':'Spanish'
       }

for key in sums:
    plt.plot(days,sums[key],label = labels[key] )
    
plt.legend()
plt.show()

In [ ]:
df_train['PageTitle'] = df_train.Page.apply(lambda x: x.split('_')[0])

In [ ]:
page_details = pd.DataFrame([i.split("_")[-3:] for i in df_train["Page"]])
page_details.columns = ["site", "access", "agent"]
page_details.describe()

In [ ]:
site_columns = page_details['site'].unique()
access_columns = page_details['access'].unique()
agents_columns = page_details['agent'].unique()


In [ ]:
lang_columns = df_train.lang.unique()

In [ ]:
title_columns = df_train['PageTitle'].unique()

In [ ]:
print(list(site_columns))

In [ ]:
print(list(access_columns))

In [ ]:
print(list(agents_columns))

In [ ]:
print(list(lang_columns))

In [ ]:
print(len(title_columns))

In [ ]:
df_train = df_train.merge(page_details, how="inner", left_index=True, right_index=True)

In [ ]:
def plot_by_feature(plot_column, name):
    df = df_train.groupby(name).sum().T
    df.index = pd.to_datetime(df.index)
    df = df.groupby(pd.TimeGrouper('M')).mean().dropna()
    df['month'] = 100*df.index.year + df.index.month
    df = df.reset_index(drop=True)
    df = pd.melt(df, id_vars=['month'], value_vars=plot_column)
    fig = plt.figure(1,figsize=[12,10])
    ax = sns.pointplot(x="month", y="value", hue=name, data=df)
    ax.set(xlabel='Year-Month', ylabel='Mean Hits')

In [ ]:
plot_by_feature(site_columns, "site")

In [ ]:
plot_by_feature(lang_columns, "lang")

In [ ]:
plot_by_feature(access_columns, "access")

In [ ]:
plot_by_feature(agents_columns, "agent")

In [ ]:
all_page = train_all.Page.copy()
train_key = train_all[['Page']].copy()
train_all = train_all.iloc[:,1:] 
train_all.head()

In [ ]:
date_idx = dict((c, i) for i, c in enumerate(train_all.columns))

In [ ]:
date_idx['2016-09-13']

In [ ]:
date_idx['2017-09-10'] - date_idx['2017-07-09']

In [ ]:
train_start = '2015-07-01'
train_end = '2017-09-10'

In [ ]:
def plot_page_traffic(df, n_series, random_state=42):
    
    sample = df.sample(n_series, random_state=random_state)
    train_series = sample.loc[:,train_start:train_end]
    labels = sample['Page'].tolist()
    
    plt.figure(figsize=(15,12))
    
    for i in range(train_series.shape[0]):
        np.log1p(pd.Series(train_series.iloc[i]).astype(np.float64)).plot(linewidth=1.5)
    
    plt.title('Page Daily Traffic')
    plt.legend(labels)
    
plot_page_traffic(df_train, 5)

In [ ]:
plot_page_traffic(df_train, 5, 2)

### Data Preprocessing
We need to create 4 sub-segments of the data:

1. Train encoding period
2. Train decoding period (train targets, 60 days)
3. Validation encoding period
4. Validation decoding period (validation targets, 60 days)

We'll do this by finding the appropriate start and end dates for each segment. Starting from the end of the data we've loaded, we'll work backwards to get validation and training prediction intervals. Then we'll work forward from the start to get training and validation encoding intervals.

In [ ]:
start_day = pd.to_datetime(train_start) 
end_day = pd.to_datetime(train_end)

steps_size = 60 
prediction_length = timedelta(steps_size)



val_start_pred = end_day - prediction_length + timedelta(days=1)
val_end_pred = end_day

train_start_pred = val_start_pred - prediction_length
train_end_pred = val_start_pred - timedelta(days=1)

In [ ]:
encoded_length = train_start_pred - start_day

train_start_encoded = start_day
train_end_encoded = train_start_encoded + encoded_length - timedelta(days=1)

val_start_encoded = train_start_encoded + prediction_length
val_end_encoded = val_start_encoded + encoded_length - timedelta(days=1)

In [7]:
print('Train encoding:', train_start_encoded, '-', train_end_encoded)
print('Train prediction:', train_start_pred, '-', train_end_pred, '\n')
print('Val encoding:', val_start_encoded, '-', val_end_encoded)
print('Val prediction:', val_start_pred, '-', val_end_pred)

print('Encoding interval:', encoded_length.days)
print('Prediction interval:', prediction_length.days)

datetime.timedelta(days=1)

In [8]:
train_df = df_train.copy()

datetime.timedelta(days=1)

In [ ]:
train_df['site'] = pd.factorize(train_df.site)[0]
train_df['lang'] = pd.factorize(train_df.lang)[0]
train_df['PageTitle'] = pd.factorize(train_df.PageTitle)[0]
train_df['access'] = pd.factorize(train_df.access)[0]
train_df['agent'] = pd.factorize(train_df.agent)[0]

In [ ]:
train_df.head()

#### Keras Data Formatting

Now that we have the time segment dates, we'll define the functions we need to extract the data in keras friendly format. Here are the steps:

Pull the time series into an array, save a date_to_index mapping as a utility for referencing into the array
Create function to extract specified time interval from all the series
Create functions to transform all the series.
Here we smooth out the scale by taking log1p and de-meaning each series using the encoder series mean, then reshape to the (n_series, n_timesteps, n_features) tensor format that keras will expect.
Note that if we want to generate true predictions instead of log scale ones, we can easily apply a reverse transformation at prediction time.

In [28]:

date_idx = pd.Series(index=pd.Index([pd.to_datetime(c) for c in train_df.columns[1:-5]]),
                          data=[i for i in range(len(train_df.columns[1:-5]))])

train_series_data = train_df[train_df.columns[1:]].values

def time_block_series_data(series_data, date_idx, start_date, end_date):
    
    indexes = date_idx[start_date:end_date]
    return series_data[:,indexes]

def encoded_transformed_series(series_data):
    
    series_data = np.log1p(np.nan_to_num(series_data))
    series_mean = series_data.mean(axis=1).reshape(-1,1) 
    series_data = series_data - series_mean
    series_data = series_data.reshape((series_data.shape[0],series_data.shape[1], 1))
    
    return series_data, series_mean

def decoded_transformed_series(series_data, encode_series_mean):
    
    series_data = np.log1p(np.nan_to_num(series_data))
    series_data = series_data - encode_series_mean
    series_data = series_data.reshape((series_data.shape[0],series_data.shape[1], 1))
    
    return series_data

In [ ]:
samples = 145000

In [ ]:
encoder_input_data = time_block_series_data(train_series_data, date_idx, 
                                            train_start_encoded, train_end_encoded)[:samples]
encoder_input_data, encode_series_mean = encoded_transformed_series(encoder_input_data)

decoder_target_data = time_block_series_data(train_series_data, date_idx, 
                                            train_start_pred, train_end_pred)[:samples]

decoder_target_data = decoded_transformed_series(decoder_target_data, encode_series_mean)

lagged_target_history = decoder_target_data[:,:-1,:1]

encoder_input_data = np.concatenate([encoder_input_data, lagged_target_history], axis=1)


### 3. Building the Model - Architecture

This convolutional architecture is a full-fledged version of the WaveNet model, designed as a generative model for audio (in particular, for text-to-speech applications). The wavenet model can be abstracted beyond audio to apply to any time series forecasting problem, providing a nice structure for capturing long-term dependencies without an excessive number of learned weights.

The core building block of the wavenet model is the dilated causal convolution layer, discussed in detail in the previous notebook of this series as well as the accompanying blog post. In summary, this style of convolution properly handles temporal flow and allows the receptive field of outputs to increase exponentially as a function of the number of layers. This structure is nicely visualized by the below diagram from the wavenet paper.

dilatedconv

The model also utilizes some other key techniques: gated activations, residual connections, and skip connections. I'll introduce and explain these techniques, then show how to implement our full-fledged WaveNet architecture in keras. The WaveNet paper diagram below details how the model's components fit together block by block into a stack of operations, so we'll use it as a handy reference as we go (note that there are slight discrepancies between the diagram and what we implement, e.g. the original WaveNet has a softmax classification rather than regression output).


**Gated Activations**
In the boxed portion of the architecture diagram, you'll notice that the dilated convolution output splits into two branches that are later recombined via element-wise multiplication. This depicts a gated activation unit, where we interpret the tanh activation branch as a learned filter and the sigmoid activation branch as a learned gate that regulates the information flow from the filter. If this reminds you of the gating mechanisms used in LSTMs or GRUs you're on point, as those models use the same style of information gating to control adjustments to their cell states.

In mathematical notation, this means we map a convolutional block's input $x$ to output $z$ via the below, where $W_f$ and $W_g$ correspond to (learned) dilated causal convolution weights:

$$ z = tanh(W_f * x) \odot \sigma(W_g * x) $$
Why use gated activations instead of the more standard ReLU activation? The WaveNet designers found that gated activations saw stronger performance empirically than ReLU activations for audio data, and this outperformance may extend broadly to time series data. Perhaps the sparsity induced by ReLU activations is not as well suited to time series forecasting as it is to other problem domains, or gated activations allow for smoother information (gradient) flow over a many-layered WaveNet architecture. However, this choice of activation is certainly not set in stone and I'd be interested to see a results comparison when trying ReLU instead. With that caveat, we'll be sticking with the gated activations in the interest of learning about the full original architecture.

**Residual and Skip Connections**
In traditional neural network architectures, a neuron layer takes direct input only from the layer that precedes it, so early layers influence deeper layers via a heirarchy of intermediate computations. In theory, this heirarchy allows the network to properly build up high-level predictive features off of lower-level/raw signals. For example, in image classification problems, neural nets start from raw pixel values, find generic geometric and textural patterns, then combine these generic patterns to construct fine-grained representations of the features that identify specific object types.

But what if lower-level signals are actually immediately useful for prediction, and may be at risk of distortion as they're passed through a complex heirarchy of computations? We could always simplify the heirarchy by using fewer layers and units, but what if we want the best of both worlds: direct, unfiltered low-level signals and nuanced heirarchical representations? One avenue for addressing this problem is provided by skip connections, which act to preserve earlier feature layer outputs as the network passes forward signals for final prediction processing. To build intuition for why we would want a mix of feature complexities in our problem domain, consider the wide range of time series drivers - there are strong and direct autoregressive components, moderately more sophisticated trend and seasonality components, and idiosyncratic trajectories that are difficult to spot with the human eye.


Residual connections are closely related to skip connections; in fact, they can be viewed as specialized, short skips further into the network (often and in our case just one layer). With residual connections, we think of mapping a network block's input to output via $x_{out} = f(x_{in}) + x_{in}$ instead of using the traditional direct mapping $x_{out} = f(x_{in})$, for some function $f$ that corresponds to the model's learned weights. This helps allow for the possibility that the model learns a mapping that acts almost as an identity function, with the input passing through nearly unchanged. In the diagram above, such connections are visualized by the rounded arrows grouped with each pair of convolutions.

Why would this be beneficial? Well, the effectiveness of residual connections is still not fully understood, but a compelling explanation is that they facilitate the use of deeper networks by allowing for more direct gradient flow in backpropagation. It's often difficult to efficienctly train the early layers of a deep network due to the length of the backpropagation chain, but residual and skip connections create an easier information highway. Intuitively, perhaps you can think of both as mechanisms for guarding against overcomputation and intermediate signal loss. You can check out the ResNet paper that originated the residual connection concept for more discussion and empirical results.

Though our architecture will be shallower than the original WaveNet (fewer convolutional blocks), we'll likely still see some benefit from introducing skip and residual connections at every block. Returning to the WaveNet architecture diagram again, you can see how the residual connection allows each block's input to bypass the convolution stage, and then adds that input to the convolution output. A final point to note is that the diagram's 1x1 convolutions are really just equivalent to (time-distributed) fully connected layers, and serve in post-processing and standardization capacities. Our setup will use layers of this style (with different filter dimensions) for post/pre-processing to facilitate our skip and residual connections, as well as for generating final prediction outputs.

**Our Architecture**

- 18 dilated causal convolutional blocks
- Preprocessing and postprocessing (time distributed) fully connected layers (convolutions with filter width 1): 
- 18 output units
- 32 filters of width 2 per block
- Exponentially increasing dilation rate with a reset (1, 2, 4, 8, 16, 32, 64, 128, 256, 1, 2, 4, 8, 16, 32, 64, 128, 256)
- Gated activations 
- Residual and skip connections
- 1 (time distributed) fully connected layers to map sum of skip outputs to final output

We'll extract the last 60 steps from the output sequence as our predicted output for training. We'll use teacher forcing again during training. Similarly to the previous notebook, we'll have a separate function that runs an inference loop to generate predictions on unseen data, iteratively filling previous predictions into the history sequence (section 4).


In [32]:
def make_model(dilation_rates):
    inp = Input(shape=(None, 1))
    x = inp
    skips = []

    for dilation_rate in dilation_rates:
        x = Conv1D(filters=16,
                   kernel_size=1,
                   padding='same',
                   activation='relu')(x) 

        x_f = Conv1D(filters=32,
                     kernel_size=2, 
                     padding='causal',
                     dilation_rate=dilation_rate)(x)

        x_g = Conv1D(filters=32,
                     kernel_size=2, 
                     padding='causal',
                     dilation_rate=dilation_rate)(x)

        z = Multiply()([Activation('tanh')(x_f),
                        Activation('sigmoid')(x_g)])

        z = Conv1D(filters=16, 
                   kernel_size=1, 
                   padding='same', 
                   activation='relu')(z)

        x = Add()([x, z])    

        skips.append(z)
            
    out = Activation('relu')(Add()(skips))
    
    out = Conv1D(filters=256, kernel_size=1, padding='same')(out)
    out = Activation('relu')(out)
    out = Dropout(rate=.5)(out)
    
    out = Conv1D(filters=128, kernel_size=1, padding='same')(out)
    out = Activation('relu')(out)
    out = Dropout(rate=.2)(out)
    
    out = Conv1D(filters=1, kernel_size=1, padding='same')(out)
    
    train_pred = Lambda(lambda x, seq: x[:,-seq:,:], 
                            arguments={'seq':60})(out)
    
    model = Model(inp, train_pred)
    return model

In [30]:
dilation_rates = [1, 2, 4, 8, 16, 32, 64, 128, 256, 1, 2, 4, 8, 16, 32, 64, 128, 256]

In [34]:
len([1, 2, 4, 8, 16, 32, 64, 128, 256, 1, 2, 4, 8, 16, 32, 64, 128, 256])

18

In [ ]:
model = make_model(dilation_rates)

In [ ]:
optimizer = Adam(lr=1e-2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss='mean_absolute_error')
    

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
checkpointer = ModelCheckpoint(filepath='web_traffic_best.hdf5',
                               verbose=1, save_best_only=True)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
              min_delta=0,
              patience=5,
              verbose=0, mode='auto')

In [ ]:
batch_size = 2048
epochs = 25

In [25]:
history = model.fit(encoder_input_data, decoder_target_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[checkpointer, learning_rate_reduction, early_stopping],
                    validation_split=0.2)

1250

In [26]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend(['Train','Valid'])

20

### Refinements

### Results

Like in the previous notebook, we'll generate predictions by running our model from section 3 in a loop, using each iteration to extract the prediction for the time step one beyond our current history then append it to our history sequence. With 60 iterations, this lets us generate predictions for the full interval we've chosen.

Recall that we designed our model to output predictions for 60 time steps at once in order to use teacher forcing for training. So if we start from a history sequence and want to predict the first future time step, we can run the model on the history sequence and take the last time step of the output, which corresponds to one time step beyond the history sequence.

In [ ]:
def predict(inputs):
    inp = inp.copy()
    preds = np.zeros((1, steps_size, 1))
    
    for i in range(steps_size):
        last_pred = model.predict(history_sequence)[0,-1,0]
        preds[0,i,0] = last_pred
        
        # add the next time step prediction to the history sequence
        inp = np.concatenate([inp, last_pred.reshape(-1,1,1)], axis=1)

    return preds

### 5. Generating and Plotting Predictions

Now we have everything we need to generate predictions for encoder (history) /target series pairs that we didn't train on (note again we're using "encoder"/"decoder" terminology to stay consistent with notebook 1 -- here it's more like history/target). We'll pull out our set of validation encoder/target series (recall that these are shifted forward in time). Then using a plotting utility function, we can look at the tail end of the encoder series, the true target series, and the predicted target series. This gives us a feel for how our predictions are doing.

In [ ]:
encoder_input_data = time_block_series_data(train_series_data, date_idx, 
                                            val_start_encoded, val_end_encoded)[:samples]
encoder_input_data, encode_series_mean = encoded_transformed_series(encoder_input_data)

decoder_target_data = time_block_series_data(train_series_data, date_idx, 
                                            val_start_pred, val_end_pred)[:samples]

decoder_target_data = decoded_transformed_series(decoder_target_data, encode_series_mean)

In [ ]:
def plot_prediction(encoder_input_data, decoder_target_data, sample_index, enc_tail_len=50):

    encode_series = encoder_input_data[sample_index:sample_index+1,:,:] 
    pred_series = predict_sequence(encode_series)
    
    encode_series = encode_series.reshape(-1,1)
    pred_series = pred_series.reshape(-1,1)   
    target_series = decoder_target_data[sample_index,:,:1].reshape(-1,1) 
    
    encode_series_tail = np.concatenate([encode_series[-enc_tail_len:],target_series[:1]])
    x_encode = encode_series_tail.shape[0]
    
    plt.figure(figsize=(10,6))   
    
    plt.plot(range(1,x_encode+1),encode_series_tail)
    plt.plot(range(x_encode,x_encode+pred_steps),target_series,color='orange')
    plt.plot(range(x_encode,x_encode+pred_steps),pred_series,color='teal',linestyle='--')
    
    plt.title('Encoder Series Tail of Length %d, Target Series, and Predictions' % enc_tail_len)
    plt.legend(['Encoding Series','Target Series','Predictions'])

Generating some plots as below, we can see that our longer time horizon predictions (60 days) are often strong and expressive. Our full-fledged model is able to effectively capture weekly seasonality patterns and long term trends, and does a very nice job adapting to the varying levels of fluctuation in each series.

Still, we can do even better! We'd benefit from increasing the sample size for training and fine-tuning our hyperparameters, but also by giving the model access to additional relevant information. So far we've only fed the model raw time series data, but it can likely benefit from the inclusion of exogenous variables such as the day of the week and the language of the webpage corresponding to each series. To see how these exogenous variables can be incorporated directly into the model check out the next notebook in this series.

If you're interested in digging even deeper into state of the art WaveNet style architectures, I also highly recommend checking out Sean Vasquez's model that was designed for this data set. He implements a customized seq2seq WaveNet architecture in tensorflow.

In [ ]:
predict_and_plot(encoder_input_data, decoder_target_data, 
                 sample_ind=16534, enc_tail_len=100)

## Futher Improvements